## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = range(1,1001)
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [4]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [5]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [6]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [7]:
# count
rdd.count()

1000

In [8]:
# first
rdd.first()

1

In [9]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [10]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [11]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [12]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([8.85207045e-01, 6.91148229e-01, 2.80582049e+00, 2.86193209e-01,
       3.77582812e+00, 5.40122899e+00, 1.21413940e+00, 7.17179163e+00,
       6.28647646e-01, 8.48127578e+00, 1.17300376e+00, 1.62364475e+00,
       7.41649168e+00, 7.84017769e+00, 8.02916152e+00, 1.07327830e+01,
       6.56652765e+00, 6.31591952e-01, 1.80026555e+01, 1.66492889e+01,
       7.64502762e+00, 1.65117375e+01, 1.52755746e+01, 1.95025554e+01,
       1.61411622e+00, 1.21032601e+01, 1.39542457e+01, 1.00041251e+01,
       4.67094135e+00, 1.37431306e+01, 2.21396910e+01, 2.52404446e+01,
       1.35005912e+01, 2.42735097e+01, 1.21818528e+01, 3.27704336e+01,
       3.20619918e+00, 2.85069177e+00, 2.70436580e+00, 3.77171418e+01,
       2.22555848e+01, 1.03423261e+01, 3.75224379e+01, 3.68860053e+01,
       1.79614512e+01, 3.80962941e+01, 2.86333602e+01, 4.66218413e+01,
       1.60392324e+01, 2.49946408e+01, 1.03323271e+01, 3.87319719e+01,
       5.13517615e+00, 6.56480345e+00, 3.17826884e+01, 2.49978251e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [13]:
price_items = sc.parallelize(sales_figures, 10)
price_items.top(5)

[945.9495901669475,
 943.1552299308834,
 927.8885150633816,
 906.7477864910529,
 900.7911440902769]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [14]:
def sales_tax(num):
    return num*.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [15]:
# perform action to retrieve rdd values
revenue_minus_tax.top(5)

[870.2736229535917,
 867.7028115364127,
 853.6574338583112,
 834.2079635717687,
 828.7278525630549]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [16]:
discounted = price_items.map(lambda x: x*.90)

In [18]:
discounted.top(5)

[851.3546311502528,
 848.8397069377951,
 835.0996635570435,
 816.0730078419476,
 810.7120296812493]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [19]:
taxed_discounted = price_items.map(lambda x: x*.90).map(sales_tax)
taxed_discounted.top(15)

[783.2462606582326,
 780.9325303827715,
 768.2916904724801,
 750.7871672145918,
 745.8550673067494,
 742.0743232149213,
 741.3366610107347,
 736.3941352873477,
 720.8809365257866,
 720.0946572560661,
 718.5047101240415,
 716.704428480209,
 715.0974495776372,
 711.913808920556,
 706.8033644406245]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [21]:
print(taxed_discounted.toDebugString())

b'(10) PythonRDD[12] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[6] at readRDDFromFile at PythonRDD.scala:262 []'


### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [23]:
def calculate(num):
    return (num, num*.90*.92)
mapped = price_items.map(calculate)
print(mapped.count())
print(mapped.top(10))

1000
[(945.9495901669475, 783.2462606582326), (943.1552299308834, 780.9325303827715), (927.8885150633816, 768.2916904724801), (906.7477864910529, 750.7871672145918), (900.7911440902769, 745.8550673067494), (896.2250280373445, 742.0743232149213), (895.3341316554765, 741.3366610107347), (889.3648976900333, 736.3941352873477), (870.629150393462, 720.8809365257866), (869.679537748872, 720.0946572560661)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [24]:
flat_mapped = price_items.flatMap(calculate)
print(flat_mapped.count())
print(flat_mapped.top(10))

2000
[945.9495901669475, 943.1552299308834, 927.8885150633816, 906.7477864910529, 900.7911440902769, 896.2250280373445, 895.3341316554765, 889.3648976900333, 870.629150393462, 869.679537748872]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [25]:
# use the filter function
selected_items = taxed_discounted.filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.count()

265

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [28]:
selected_items.reduce(lambda x, y: y + x)

125788.90695179637

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [29]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(48, 310.63476840061395),
 (47, 346.33285036790284),
 (26, 355.96551366646815),
 (19, 359.6093300845856),
 (12, 353.9959978467905),
 (48, 355.3533167741962),
 (45, 384.28618122030446)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [36]:
# calculate how much each user spent
user_totals = sales_data.reduceByKey(lambda x, y: x+y)
user_totals.top(5)

[(50, 2323.6239494525207),
 (49, 571.6162689278862),
 (48, 952.0625719325195),
 (47, 1637.636018830537),
 (46, 2798.5353277104914)]

In [37]:
# sort the users from highest to lowest spenders
user_totals.sortBy(lambda x: x[1], ascending=False).collect()

[(43, 5427.994106591083),
 (18, 5249.727977640738),
 (6, 4263.422995996272),
 (21, 4107.267552203422),
 (24, 3882.856759257869),
 (31, 3573.4109785654164),
 (14, 3511.086191024955),
 (37, 3502.887099799348),
 (16, 3438.0923259826145),
 (26, 3374.428767339362),
 (36, 3200.7778191516372),
 (34, 3068.739082485463),
 (11, 3063.5923247745145),
 (29, 3021.9868667266283),
 (12, 2852.02451253554),
 (46, 2798.5353277104914),
 (30, 2755.1039822794705),
 (19, 2747.998873548456),
 (42, 2737.9098888340695),
 (7, 2713.476855059901),
 (32, 2668.9858616992497),
 (22, 2605.5070766419726),
 (45, 2591.6050733320744),
 (23, 2581.14716972121),
 (44, 2572.0418469606984),
 (15, 2494.4360168326166),
 (13, 2439.77199930373),
 (25, 2434.458186937397),
 (35, 2359.299653660434),
 (9, 2351.537765732103),
 (50, 2323.6239494525207),
 (2, 2051.3963343147097),
 (39, 1974.4974605933407),
 (27, 1938.64247276913),
 (20, 1885.669042656051),
 (38, 1854.458623824376),
 (28, 1840.4618375476925),
 (17, 1793.9920683916614),
 (

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [40]:
total_items = sales_data.countByKey()
sorted(total_items.items(), key=lambda x: x[1], reverse=True)

[(22, 15),
 (27, 11),
 (11, 11),
 (4, 10),
 (40, 9),
 (31, 8),
 (45, 8),
 (21, 8),
 (5, 7),
 (35, 7),
 (30, 6),
 (28, 6),
 (7, 6),
 (20, 6),
 (19, 6),
 (3, 6),
 (37, 6),
 (47, 6),
 (50, 5),
 (14, 5),
 (34, 5),
 (24, 5),
 (17, 5),
 (48, 5),
 (26, 5),
 (25, 5),
 (32, 5),
 (9, 5),
 (2, 5),
 (49, 5),
 (1, 4),
 (46, 4),
 (44, 4),
 (16, 4),
 (13, 4),
 (38, 4),
 (18, 4),
 (29, 4),
 (33, 4),
 (12, 3),
 (8, 3),
 (23, 3),
 (10, 3),
 (15, 3),
 (42, 3),
 (36, 3),
 (41, 3),
 (39, 1),
 (43, 1),
 (6, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
